# 신경망 모델 구성하기 
- 신경망은 데이터에 대한 연산을 수행하는 계층(layer)과 / 모듈(module)로 구성 
- torch.nn은 신경망을 구성하는데 필요한 모든 구성요소를 제공 
- 파이토치의 모든 모듈은 nn.module의 subclass 이다.

### FashionMNIST 데이터셋의 이미지들을 분류하는 신경망 구성 

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
## torch.cuda 를 사용할 수 있는지 확인하고 그렇지 않으면 CPU 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 클래스 정의
1. 신경망 모델을 nn.Module의 하위클래스로 정의
2. ____init____ 에서 신경망 계층들을 초기화 
3. nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력데이터에 대한 연산 구현

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() 
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
# 신경망(NN)의 instance를 생성하고 이를 device로 이동한 뒤 구조 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### 모델을 사용하기 위해 입력 데이터 전달
### -> 일부 백그라운드 연산들과 함께 forward 실행 
### -> model.forward()를 직접 호출하면 안된다!!!! 

- 모델에 입력을 호출하면 class에 대한 raw 예측값이 있는 10-차원 텐서가 반환 
- raw 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 에측 확률을 얻는다

In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) 
y_pred = pred_probab.argmax(1)
print(f"Predictes class : {y_pred}")

Predictes class : tensor([1])


## 모델 계층(Layer) 
- FashionMNIST 모델의 계층에서 28x28 크기의 이미지 3개로 구성된 미니배치를 가져옴 
- 미니배치가 신경망을 통과할 때 어떤 일이 발생하는지 확인

In [23]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [24]:
# nn.Flatten
# nn.Flatten 계층을 초기화화여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환 
# dim = 0 의 미니배치 차원은 유지

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [40]:
# nn.Linear
# nn.Linear 계층 (선형 계층)은 저장된 weight 와 bias 을 사용하여 
# 입력에 linear transformation을 적용하는 모듈

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [41]:
# nn.ReLU
# nn.ReLU 계층 (비선형 활성화 함수)는 모델의 입력, 출력 사이에 복잡한 관계(mapping)를 만듬
# ReLU는 선형 변환 후에 적용되어, nonlinearity을 도입하고 다양한 현상을 학습할 수 있도록 한다.

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}\n\n")

Before ReLU: tensor([[-0.4607, -0.2667, -0.0671, -0.3316,  0.3956, -0.6062, -0.1584, -0.0325,
         -0.3378, -0.4784, -0.4134, -0.3081, -0.0929, -0.2360,  0.4872,  0.0712,
          0.4033, -0.0088,  0.1807,  0.1286],
        [-0.7877, -0.2144, -0.2403,  0.3330,  0.2480, -0.2055, -0.1697, -0.2095,
         -0.2691, -0.5553, -0.3738, -0.5517,  0.2990, -0.1514,  0.1281,  0.2425,
          0.4382, -0.1303,  0.4389, -0.0710],
        [-0.5316, -0.4713, -0.1605, -0.1717,  0.3154, -0.1773, -0.1567,  0.1298,
         -0.2905, -0.3048, -0.0735, -0.5940, -0.1182, -0.5014,  0.4073,  0.1175,
          0.6152, -0.0009,  0.2409, -0.1935]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.3956, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4872, 0.0712, 0.4033, 0.0000,
         0.1807, 0.1286],
        [0.0000, 0.0000, 0.0000, 0.3330, 0.2480, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.2990, 0.0000, 0.128

In [42]:
# nn.Sequential
# nn.Sequential은 순서를 갖는 모듈의 컨테이너 
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달 
# sequential container(순차 컨테이너) 사용하여 신경망을 빠르게 만들 수 있다.

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [43]:
# nn.Softmax 
# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환
# logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 scale 된다. 
# dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.DataLoader

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 매개변수 
- 신경망 내부의 많은 계층들은 parameterize (매개변수화) 된다.
- 학습 중에 최적화되는 가중치와 편향과 연관이 된다는 의미! 
- nn.Module을 상속하면 객체 내부의 모든 필드들이 자동으로 추적 된다.
- 모델의 prarmeters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다. 

In [45]:
# 각 매개변수들을 순회(iterate)하며, 매개변수의 크기와 값을 출력 
print("Model structure : ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size : {param.size()} | Values : {param[:2]} \n")

Model structure :  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[-0.0176, -0.0117, -0.0206,  ..., -0.0084,  0.0333,  0.0314],
        [-0.0176,  0.0050,  0.0275,  ...,  0.0338, -0.0346,  0.0261]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0182,  0.0345], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0398, -0.0214, -0.0168,  ...,  0.0224, -0.0080, -0.0315],
        [ 0.0214, -0.0434,  0.0184,  ...,  0.0082,  0.0353,  0.0068]],
       grad_fn=<SliceBackward>) 

Layer: linear_r